🎯 AUC-ROC Score: 0.8757
🎯 AUC-PR Score: 0.6035

Class 0: FPR = 0.4698, FNR = 0.0112
Class 1: FPR = 0.0000, FNR = 1.0000
Class 2: FPR = 0.0194, FNR = 0.4609

In [10]:
!pip install transformers datasets torch scikit-learn


In [11]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from torch.nn import CrossEntropyLoss
from transformers import RobertaTokenizer, RobertaForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight


In [17]:
import pandas as pd


df = pd.read_csv("/kaggle/input/flabels-1/final_labels.csv")  

# Display available columns
print("Available columns:", df.columns)


Available columns: Index(['entry_id', 'link_id', 'parent_id', 'entry_utc', 'subreddit', 'author',
       'body', 'image', 'label_date', 'week', 'group', 'sheet_order',
       'level_1', 'level_2', 'level_3', 'strength', 'highlight', 'split'],
      dtype='object')


In [18]:
# Drop rows where 'body' or 'level_2' is missing
df = df.dropna(subset=["body", "level_2"]).reset_index(drop=True)


print(f"Dataset size after cleaning: {len(df)}")


Dataset size after cleaning: 6555


In [19]:
# Define label mapping
label_mapping = {
    "None_of_the_categories": 0,  # Non-Toxic
    "Counter_speech": 0,  # Non-Toxic
    "Treatment": 1,  # Neutral
    "Derogation": 2,  # Toxic
    "Nonmisogynistic_personal_attack": 2,  # Toxic
    "Misogynistic_personal_attack": 2,  # Toxic
    "Misogynistic_pejorative": 2  # Toxic
}

# Apply mapping to the 'level_2' column
df["label"] = df["level_2"].map(label_mapping)


df = df.dropna(subset=["label"]).reset_index(drop=True)

# Convert labels to integer type
df["label"] = df["label"].astype(int)

# Print unique labels after mapping
print("Unique labels after mapping:", df["label"].unique())


Unique labels after mapping: [0 2 1]


In [20]:
from sklearn.model_selection import train_test_split

# Extract text and labels
texts = df["body"].tolist() 
labels = df["label"].tolist()  

# Split data (80% Train, 20% Validation)
train_texts, val_texts, train_labels, val_labels = train_test_split(
    texts, labels, test_size=0.2, random_state=42, stratify=labels
)

print(f"Training samples: {len(train_texts)}, Validation samples: {len(val_texts)}")


Training samples: 5244, Validation samples: 1311


In [7]:
label_mapping = {
    "None_of_the_categories": 0,  # Non-Toxic
    "Counter_speech": 0,  # Non-Toxic
    "Treatment": 1,  # Neutral
    "Derogation": 2,  # Toxic
    "Nonmisogynistic_personal_attack": 2,  # Toxic
    "Misogynistic_personal_attack": 2,  # Toxic
    "Misogynistic_pejorative": 2  # Toxic
}

df["label"] = df["label"].map(label_mapping)
df = df.dropna(subset=["label"]) 


In [21]:
from transformers import RobertaTokenizer

# Load RoBERTa tokenizer
tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# Tokenize training & validation texts
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
val_encodings = tokenizer(val_texts, truncation=True, padding=True, max_length=512)


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

In [22]:
import torch
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Create PyTorch datasets
train_dataset = CustomDataset(train_encodings, train_labels)
val_dataset = CustomDataset(val_encodings, val_labels)


In [23]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight

# Compute class weights
class_weights = compute_class_weight(
    class_weight="balanced", 
    classes=np.unique(train_labels), 
    y=train_labels
)

# Convert to Tensor
class_weights = torch.tensor(class_weights, dtype=torch.float).to("cuda")

print("Class weights:", class_weights)


Class weights: tensor([ 0.3758, 21.3171,  3.4207], device='cuda:0')


In [24]:
from transformers import RobertaForSequenceClassification

# Load RoBERTa model with 3 output classes (Non-Toxic, Neutral, Toxic)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=3)
model.to(device)


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [29]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="/kaggle/working/results",
    evaluation_strategy="epoch",  # Evaluate after every epoch
    save_strategy="epoch",  # Save model after every epoch
    learning_rate=5e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=10,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,  # Log every 10 steps
    load_best_model_at_end=True,
    report_to="none",  # Disable reporting to WandB or TensorBoard
    logging_strategy="epoch"  # Log at the end of every epoch
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [30]:
import numpy as np
from sklearn.metrics import accuracy_score

# Define metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc}

# Define Trainer with logging
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

# Train model
trainer.train()


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.395500,0.398384,0.887872
2,0.358800,0.356983,0.911518
3,0.342300,0.345308,0.911518
4,0.318600,0.294939,0.926011
5,0.296900,0.315977,0.918383
6,0.256600,0.291073,0.929062
7,0.214700,0.300963,0.929825
8,0.185100,0.316749,0.923722
9,0.155000,0.334393,0.916095
10,0.139300,0.353737,0.922197


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked t

TrainOutput(global_step=3280, training_loss=0.26626571562232043, metrics={'train_runtime': 3156.3944, 'train_samples_per_second': 16.614, 'train_steps_per_second': 1.039, 'total_flos': 1.379766762565632e+16, 'train_loss': 0.26626571562232043, 'epoch': 10.0})

In [27]:
# Save model
model.save_pretrained("fine_tuned_roberta")
tokenizer.save_pretrained("fine_tuned_roberta")

print("Fine-tuning complete! Model saved.")


Fine-tuning complete! Model saved.


## EVALUATION

In [31]:
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, average_precision_score, confusion_matrix

# Get predictions
predictions = trainer.predict(val_dataset)
logits = predictions.predictions
true_labels = predictions.label_ids

# Convert logits to probabilities (Softmax for multi-class)
probabilities = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()

# Predicted labels (highest probability class)
pred_labels = np.argmax(probabilities, axis=1)


/usr/local/lib/python3.10/dist-packages/torch/nn/parallel/_functions.py:71: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


In [34]:
from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.preprocessing import label_binarize
import numpy as np

# Convert true labels to one-hot encoding
num_classes = len(set(true_labels))
true_labels_ohe = label_binarize(true_labels, classes=list(range(num_classes)))

# Compute AUC-ROC (multi-class mode)
auc_roc = roc_auc_score(true_labels_ohe, probabilities, multi_class='ovr')

# Compute AUC-PR (macro average over classes)
auc_pr = average_precision_score(true_labels_ohe, probabilities, average='macro')

print(f"🎯 AUC-ROC Score: {auc_roc:.4f}")
print(f"🎯 AUC-PR Score: {auc_pr:.4f}")


🎯 AUC-ROC Score: 0.8757
🎯 AUC-PR Score: 0.6035


In [36]:
from sklearn.metrics import confusion_matrix
import numpy as np

# Compute confusion matrix
cm = confusion_matrix(true_labels, pred_labels)
num_classes = cm.shape[0]

# Compute FPR & FNR for each class
fpr_list, fnr_list = [], []

for i in range(num_classes):
    tp = cm[i, i]  # True Positives for class i
    fn = np.sum(cm[i, :]) - tp  # False Negatives (missed class i)
    fp = np.sum(cm[:, i]) - tp  # False Positives (wrongly classified as class i)
    tn = np.sum(cm) - (tp + fn + fp)  # True Negatives (everything else)

    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0  # Avoid division by zero
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0

    fpr_list.append(fpr)
    fnr_list.append(fnr)

# Print FPR & FNR per class
for i in range(num_classes):
    print(f"Class {i}: FPR = {fpr_list[i]:.4f}, FNR = {fnr_list[i]:.4f}")


Class 0: FPR = 0.4698, FNR = 0.0112
Class 1: FPR = 0.0000, FNR = 1.0000
Class 2: FPR = 0.0194, FNR = 0.4609
